# Linear Regression - ECommerce

Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [20]:
# Lets start a Spark Session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [21]:
# Import the Linear Regression library

from pyspark.ml.regression import LinearRegression

## Read the dataset

In [22]:
# We will now read the dataset

data = spark.read.csv("resources/Ecommerce_Customers.csv", inferSchema=True, header=True)

# Print the schema of the data frame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [23]:
# View the initial rows of the data
data.show(10)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [24]:
# Check the first row
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [25]:
# Check the individual column values from the first row
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up the dataframe for Machine Learning

A few things we need to do before Spark can accept the data! It needs to be in the form of two columns 
("label","features")

In [26]:
# Import VectorAssembler and Vectors

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [27]:
# Check the columns
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [28]:
# We will use the numerical columns only for now

assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 
                                       'Length of Membership', 'Yearly Amount Spent'], 
                            outputCol="features")

# Now we will transform the data

output = assembler.transform(data)

In [29]:
# If we do a printSchema we will see a feautures column got added which is actually a dense vector

output.printSchema()

# See the first row
output.head()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))

In [30]:
# Final data for regression

final_data = output.select("features", 'Yearly Amount Spent')

In [31]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## Perform the Regression

In [32]:
# Perform the train, test split

train_data, test_data = final_data.randomSplit([0.7, 0.3])

train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean| 498.70051385059253|
| stddev|  78.37051924021307|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [33]:
# Create the Linear Regression Model Object

lr = LinearRegression(labelCol='Yearly Amount Spent')

# Fit the model to the data and call this model
lrModel = lr.fit(train_data,)

In [34]:
# Get the coefficients and intercept

print(f"coefficients : {lrModel.coefficients}")
print("\n")
print(f"Intercept : {lrModel.intercept}")

coefficients : [3.2342318017582547e-12,4.871090514225532e-12,1.4286814154003213e-14,7.658542575150552e-12,0.9999999999998747]


Intercept : -1.306753173580155e-10


In [35]:
# Now we will evaluate this model 

test_results = lrModel.evaluate(test_data)

In [36]:
# Get the residuals, RMSE and Adjusted R2

test_results.residuals.show()

print(f"RMSE : {test_results.rootMeanSquaredError}")
print(f"Adjusted R2 : {test_results.r2adj}")

+--------------------+
|           residuals|
+--------------------+
|-6.82121026329696...|
|1.364242052659392...|
|-2.16004991671070...|
|-5.11590769747272...|
|9.663381206337363...|
|6.821210263296962...|
|-9.09494701772928...|
|2.728484105318784...|
|4.547473508864641...|
|-7.95807864051312...|
|-5.11590769747272...|
|-5.11590769747272...|
|-1.13686837721616...|
|-6.25277607468888...|
|9.094947017729282...|
|-7.38964445190504...|
|-2.21689333557151...|
|-1.13686837721616...|
|-2.27373675443232...|
|-6.25277607468888...|
+--------------------+
only showing top 20 rows

RMSE : 1.2412063993220555e-12
Adjusted R2 : 1.0


In [37]:
# Now if we have to run these as predictions on an unseen data

unlabeled_data = test_data.select('features')

predictions = lrModel.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 282.4712457199152|
|[30.7377203726281...| 461.7807421962285|
|[30.8162006488763...|266.08634094847116|
|[30.8364326747734...| 467.5019004269901|
|[30.9716756438877...|494.63860975689175|
|[31.0472221394875...| 392.4973991890207|
|[31.1280900496166...| 557.2526867470556|
|[31.2834474760581...| 591.7810894256647|
|[31.4459724827577...| 484.8769649351281|
|[31.4474464941278...| 418.6027420952248|
|[31.5147378578019...| 489.8124879964619|
|[31.5171218025062...| 275.9184206503862|
|[31.5761319713222...| 541.2265839893284|
|[31.6253601348306...| 376.3369007569248|
|[31.6548096756927...| 475.2634237275476|
|[31.7242025238451...|503.38788728796123|
|[31.8164283341993...| 501.1224915036586|
|[31.8279790554652...|440.00274754694266|
|[31.8627411090001...| 556.2981411740469|
|[31.8745516945853...| 392.2852442462681|
+--------------------+------------